In [11]:
import Estimate_Covariance_Matrix as ECM
from portfolio_choice_functions import ew_implement
from prepare_portfolio_data import define_important_dates, create_date_ranges
from Main import settings, pf_set, features
import pandas as pd
from datetime import datetime
from pandas.tseries.offsets import MonthEnd
from Prepare_Data import process_risk_free_rate, load_and_filter_market_returns_test, wealth_func
import os
import pickle

import prepare_portfolio_data
import data_run_files
import Estimate_Covariance_Matrix
import Prepare_Data
import portfolio_choice_functions
from portfolio_choice_functions import w_fun
import numpy as np
from General_Functions import initial_weights_new, pf_ts_fun
from portfolio_choice_functions import w_fun, ew_implement

## Henter de relevante data:

In [2]:
# Kald main() for at få fat i barra_cov
barra_cov = ECM.main()
print(barra_cov)

All missing excludes 0.60% of the observations
Filen er indlæst og filtreret succesfuldt.
No size screen
Turnover wo addition/deletion rule: 0.19%
Turnover w  addition/deletion rule: 33.72%


Processerer datoer: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 72/72 [00:00<00:00, 91.69it/s]


{'2017-11-30': {'fct_load':        BusEq  Chems  Durbl  Enrgy  Hlth  Manuf  Money  NoDur  Other  Shops  \
id                                                                           
10104    1.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0    0.0   
10107    1.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0    0.0   
10145    0.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    1.0    0.0   
11308    0.0    0.0    0.0    0.0   0.0    0.0    0.0    1.0    0.0    0.0   
11762    0.0    0.0    0.0    0.0   0.0    1.0    0.0    0.0    0.0    0.0   
...      ...    ...    ...    ...   ...    ...    ...    ...    ...    ...   
91937    0.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0    0.0   
92157    1.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0    0.0   
92602    0.0    0.0    0.0    0.0   0.0    0.0    0.0    1.0    0.0    0.0   
92611    0.0    0.0    0.0    0.0   0.0    0.0    1.0    0.0    0.0    0.0   
92655    0.0    0.0    0.0    0.0   

In [3]:
wealth_end = pf_set["wealth"]
end = settings["split"]["test_end"]
market_path = "./data_test/market_returns_test.csv"
rente_path = "Data/ff3_m.csv"
risk_free = process_risk_free_rate(rente_path)
market_test = load_and_filter_market_returns_test(market_path)

wealth = wealth_func(wealth_end, end, market_test, risk_free)
wealth = wealth.dropna(subset=['wealth'])
print(wealth)

Filen er indlæst og filtreret succesfuldt.
            eom        wealth    mu_ld1
1002 2009-12-31  1.468331e+09 -0.038043
1003 2010-01-31  1.414519e+09  0.034614
1004 2010-02-28  1.465237e+09  0.064595
1005 2010-03-31  1.566419e+09  0.019248
1006 2010-04-30  1.597162e+09 -0.080946
...         ...           ...       ...
1165 2023-07-31  9.975058e+09 -0.019997
1166 2023-08-31  9.779494e+09 -0.048173
1167 2023-09-30  9.330037e+09 -0.029172
1168 2023-10-31  9.065579e+09  0.093442
1169 2023-11-30  1.000000e+10       NaN

[168 rows x 3 columns]


# 1/N - base case

#R-koden er: 
ew <- chars %>% ew_implement(dates = dates_oos, wealth = wealth)

Vi skal bruge dates_oos, wealth og chars dem henter vi her

In [4]:
first_cov_date, hp_years, start_oos = define_important_dates(barra_cov, settings)

In [5]:
date_ranges = create_date_ranges(settings, first_cov_date, start_oos, hp_years)
dates_oos = date_ranges["dates_oos"]
print(dates_oos)

DatetimeIndex(['2019-12-31', '2020-01-31', '2020-02-29', '2020-03-31',
               '2020-04-30', '2020-05-31', '2020-06-30', '2020-07-31',
               '2020-08-31', '2020-09-30', '2020-10-31', '2020-11-30',
               '2020-12-31', '2021-01-31', '2021-02-28', '2021-03-31',
               '2021-04-30', '2021-05-31', '2021-06-30', '2021-07-31',
               '2021-08-31', '2021-09-30', '2021-10-31', '2021-11-30',
               '2021-12-31', '2022-01-31', '2022-02-28', '2022-03-31',
               '2022-04-30', '2022-05-31', '2022-06-30', '2022-07-31',
               '2022-08-31', '2022-09-30', '2022-10-31', '2022-11-30',
               '2022-12-31', '2023-01-31', '2023-02-28', '2023-03-31',
               '2023-04-30', '2023-05-31', '2023-06-30', '2023-07-31',
               '2023-08-31', '2023-09-30', '2023-10-31'],
              dtype='datetime64[ns]', freq='ME')


In [6]:
wealth_end = pf_set["wealth"]
end = settings["split"]["test_end"]
market_path = "./data_test/market_returns_test.csv"
rente_path = "Data/ff3_m.csv"
risk_free = process_risk_free_rate(rente_path)
market_test = load_and_filter_market_returns_test(market_path)

wealth = wealth_func(wealth_end, end, market_test, risk_free)
wealth = wealth.dropna(subset=['wealth'])
print(wealth)

Filen er indlæst og filtreret succesfuldt.
            eom        wealth    mu_ld1
1002 2009-12-31  1.468331e+09 -0.038043
1003 2010-01-31  1.414519e+09  0.034614
1004 2010-02-28  1.465237e+09  0.064595
1005 2010-03-31  1.566419e+09  0.019248
1006 2010-04-30  1.597162e+09 -0.080946
...         ...           ...       ...
1165 2023-07-31  9.975058e+09 -0.019997
1166 2023-08-31  9.779494e+09 -0.048173
1167 2023-09-30  9.330037e+09 -0.029172
1168 2023-10-31  9.065579e+09  0.093442
1169 2023-11-30  1.000000e+10       NaN

[168 rows x 3 columns]


In [7]:
file_path_usa_dsf = "./Data/usa_dsf.parquet"
file_path_usa = "./Data/usa_rvol.parquet"
file_path_id_test = "./data_test/top_5_percent_ids.csv"
file_path_market_returns = "./Data/market_returns.csv"
output_path_usa_dsf = "./data_test/usa_dsf_test.parquet"
output_path_usa = "./data_test/usa_test.parquet"
output_path_market_returns = "data_test/market_returns_test.csv"
file_path_world_ret = "./Data/world_ret_monthly.csv"
start_date = "2010-01-31"
end_date = "2023-11-30"

output_path = "./data_test/"

file_path_usa_test = "./data_test/usa_test.parquet"
daily_file_path = "./data_test/usa_dsf_test.parquet"
file_path_world_ret = "./data_test/world_ret_test.csv"
risk_free_path = "./data_test/risk_free_test.csv"
market_path = "./data_test/market_returns_test.csv"

rente_path = "Data/ff3_m.csv"


# Kører funktioner fra data_run_files
risk_free = data_run_files.process_risk_free_rate(rente_path, start_date)
h_list = [1]  # Horisonter
data_ret = data_run_files.monthly_returns(risk_free, h_list, output_path_usa)
chars, daily = Prepare_Data.process_all_data(file_path_usa_test, daily_file_path, file_path_world_ret, risk_free_path, market_path)
get_from_path_model = "./data_test/"
chars = prepare_portfolio_data.add_return_predictions(chars, settings, get_from_path_model)
lambda_list = prepare_portfolio_data.create_lambda_list(chars)

Filtrering udført. Antal rækker efter filtrering: 179
Fil gemt som ./data_test/risk_free_test.csv
All missing excludes 0.64% of the observations
All missing excludes 0.60% of the observations
Filen er indlæst og filtreret succesfuldt.
No size screen
Turnover wo addition/deletion rule: 0.19%
Turnover w  addition/deletion rule: 33.72%


In [8]:
def initial_weights_new(data, w_type, udf_weights=None):
    """
    Beregner initiale vægte baseret på w_type.
    
    Parametre:
      - data: Pandas DataFrame med mindst kolonnerne 'id', 'eom' og (afhængigt af w_type) 'me'
      - w_type: str, en af "vw", "ew", "rand_pos" eller "udf"
      - udf_weights: DataFrame med brugerdefinerede vægte (bruges, hvis w_type == "udf")
      
    Returnerer:
      - pf_w: DataFrame med kolonnerne 'id', 'eom', 'w_start' og 'w'
    """
    # Lav en kopi af data for at undgå sideeffekter
    df = data.copy()
    
    if w_type == "vw":
        # For hver eom beregnes w_start som me divideret med summen af me i gruppen
        df['w_start'] = df.groupby('eom')['me'].transform(lambda x: x / x.sum())
        pf_w = df[['id', 'eom', 'w_start']].copy()
        
    elif w_type == "ew":
        # For hver eom sættes w_start = 1 / antal elementer i gruppen
        df['w_start'] = df.groupby('eom')['id'].transform(lambda x: 1 / len(x))
        pf_w = df[['id', 'eom', 'w_start']].copy()
        
    elif w_type == "rand_pos":
        # Vælg kun de relevante kolonner
        pf_w = df[['id', 'eom']].copy()
        # Generér tilfældige tal for hver række
        pf_w['w_start'] = np.random.uniform(0, 1, size=len(pf_w))
        # Normalisér inden for hver eom, så summen af w_start = 1
        pf_w['w_start'] = pf_w.groupby('eom')['w_start'].transform(lambda x: x / x.sum())
        
    elif w_type == "udf":
        if udf_weights is None:
            raise ValueError("udf_weights skal angives, når w_type er 'udf'")
        # Merge udf_weights med data på id og eom
        # Her antages det, at udf_weights indeholder kolonnen 'w_start'
        pf_w = pd.merge(data[['id', 'eom']], udf_weights, on=['id', 'eom'], how='left')
        
    else:
        raise ValueError("Ugyldigt w_type angivet.")
    
    # For alle rækker, hvor eom ikke er den mindste (tidligste) dato, sættes w_start til NaN
    min_eom = pf_w['eom'].min()
    pf_w.loc[pf_w['eom'] != min_eom, 'w_start'] = np.nan
    
    # Tilføj kolonnen 'w' og sæt den til NaN for alle rækker
    pf_w['w'] = np.nan
    
    return pf_w

def w_fun(data, dates, w_opt, wealth):
    # Opret et DataFrame med slutmåned (eom) og forrige slutmåned (eom_prev)
    dates_df = pd.DataFrame({'eom': dates})
    dates_df['eom_prev'] = dates_df['eom'].shift(1)
    
    # Udregn initiale vægte og fjern kolonnen 'w'
    w = initial_weights_new(data, w_type="vw").drop(columns=['w'])
    
    # Join: tilføj w_opt på (id, eom)
    w = pd.merge(w, w_opt, on=['id', 'eom'], how='left')
    
    # Join med dates_df for at få eom_prev
    w = pd.merge(w, dates_df, on='eom', how='left')
    
    # Tilføj 'tr_ld1' fra data til w_opt (join på id og eom)
    data_subset = data[['id', 'eom', 'tr_ld1']]
    w_opt = pd.merge(w_opt, data_subset, on=['id', 'eom'], how='left')
    
    # Tilføj 'mu_ld1' fra wealth (join på eom)
    wealth_subset = wealth[['eom', 'mu_ld1']]
    w_opt = pd.merge(w_opt, wealth_subset, on='eom', how='left')
    
    # Opdater 'w' ift. afkastjustering: w = w*(1+tr_ld1)/(1+mu_ld1)
    w_opt['w'] = w_opt['w'] * (1 + w_opt['tr_ld1']) / (1 + w_opt['mu_ld1'])
    
    # Omdøb kolonner: 'w' -> 'w_prev' og 'eom' -> 'eom_prev'
    w_opt = w_opt.rename(columns={'w': 'w_prev', 'eom': 'eom_prev'})
    
    # Join w med w_opt på (id, eom_prev)
    w = pd.merge(w, w_opt, left_on=['id', 'eom_prev'], right_on=['id', 'eom_prev'], how='left')
    
    # For alle rækker, hvor eom ikke er den mindste dato, sættes w_start = w_prev
    min_eom = w['eom'].min()
    w.loc[w['eom'] != min_eom, 'w_start'] = w['w_prev']
    
    # Udfyld eventuelle manglende værdier i w_start med 0
    w['w_start'] = w['w_start'].fillna(0)
    
    # Fjern de midlertidige kolonner
    w = w.drop(columns=['eom_prev', 'w_prev'])
    
    return w

def pf_ts_fun(weights, data, wealth, gam):
    # Join weights med relevante data-kolonner (id, eom, ret_ld1, pred_ld1, lambda)
    data_subset = data[['id', 'eom', 'ret_ld1', 'pred_ld1', 'lambda']]
    comb = pd.merge(weights, data_subset, on=['id', 'eom'], how='left')
    
    # Join med wealth for at tilføje 'wealth'-kolonnen (join på eom)
    wealth_subset = wealth[['eom', 'wealth']]
    comb = pd.merge(comb, wealth_subset, on='eom', how='left')
    
    # Aggregér porteføljestatistikker pr. eom
    def agg_func(g):
        inv = np.sum(np.abs(g['w']))
        shorting = np.sum(np.abs(g.loc[g['w'] < 0, 'w']))
        turnover = np.sum(np.abs(g['w'] - g['w_start']))
        r = np.sum(g['w'] * g['ret_ld1'])
        # For tc antages det, at der er et unikt wealth pr. eom
        unique_wealth = g['wealth'].unique()[0] if len(g['wealth'].unique()) > 0 else np.nan
        tc = (unique_wealth / 2) * np.sum(g['lambda'] * ((g['w'] - g['w_start'])**2))
        return pd.Series({
            'inv': inv,
            'shorting': shorting,
            'turnover': turnover,
            'r': r,
            'tc': tc
        })
    
    pf = comb.groupby('eom').apply(agg_func).reset_index()
    
    # Udregn eom_ret som eom plus 1 måned (forudsætter at eom er en datetime)
    pf['eom'] = pd.to_datetime(pf['eom'])
    pf['eom_ret'] = pf['eom'] + pd.DateOffset(months=1)
    
    # Fjern den oprindelige eom-kolonne, hvis ønsket
    pf = pf.drop(columns=['eom'])
    
    return pf

def ew_implement(data, wealth, dates):
    # Filtrer data, så kun gyldige rækker med eom i dates bevares
    filtered = data[(data['valid'] == True) & (data['eom'].isin(dates))].copy()
    
    # Beregn antallet af rækker pr. eom
    counts = filtered.groupby('eom').size().reset_index(name='n')
    filtered = pd.merge(filtered, counts, on='eom', how='left')
    
    # Udregn optimale vægte: for hver (id, eom) sættes w = 1/n
    ew_opt = filtered[['id', 'eom']].copy()
    ew_opt['w'] = 1 / filtered['n']
    
    # Beregn vægte vha. w_fun
    weights_data = data[(data['eom'].isin(dates)) & (data['valid'] == True)].copy()
    ew_w = w_fun(weights_data, dates, ew_opt, wealth)
    
    # Beregn portefølje-statistikker vha. pf_ts_fun og tilføj porteføljetypen "1/N"
    ew_pf = pf_ts_fun(ew_w, data, wealth, gam=pf_set['gamma_rel'])
    ew_pf['type'] = "1/N"
    
    return {"w": ew_w, "pf": ew_pf}


In [13]:
result = ew_implement(data=chars, wealth=wealth, dates=dates_oos, pf_set = pf_set)


In [14]:
result

{'w':          id        eom   w_start         w    tr_ld1    mu_ld1
 0     10104 2019-12-31  0.008246  0.005814       NaN       NaN
 1     10104 2020-01-31  0.005791  0.005814 -0.005238 -0.001351
 2     10104 2020-02-29  0.005959  0.005814 -0.056915 -0.079921
 3     10104 2020-03-31  0.006595  0.005814 -0.022915 -0.138610
 4     10104 2020-04-30  0.005649  0.005814  0.102269  0.134423
 ...     ...        ...       ...       ...       ...       ...
 8079  92655 2023-06-30  0.005383  0.005814 -0.010165  0.069017
 8080  92655 2023-07-31  0.005906  0.005814  0.053142  0.036647
 8081  92655 2023-08-31  0.005583  0.005814 -0.058868 -0.019997
 8082  92655 2023-09-30  0.006488  0.005814  0.062111 -0.048173
 8083  92655 2023-10-31  0.006359  0.005814  0.061804 -0.029172
 
 [8084 rows x 6 columns],
 'pf':     inv  shorting  turnover         r        tc    eom_ret type
 0   1.0       0.0  0.731222 -0.012589  0.009614 2020-01-31  1/N
 1   1.0       0.0  0.055972 -0.082015  0.000114 2020-02-29  1/

In [ ]:
def tpf_implement(data, cov_list, wealth, dates, gam):
    """
    Beregner tangency portfolio weights og porteføljestatistikker.
    
    Parametre:
      - data: DataFrame med mindst kolonnerne 'id', 'eom', 'me', 'tr_ld1', 'pred_ld1' og 'valid'
      - cov_list: dictionary (eller lignende) hvor nøglen er en dato, og værdien et objekt med en metode create_cov(ids=...)
      - wealth: DataFrame med kolonnerne 'eom', 'mu_ld1' og 'wealth'
      - dates: liste over datoer (samme format som data['eom'])
      - gam: risikoaversion parameter
      
    Returnerer:
      - en dictionary med nøglerne "w" (de faktiske vægte) og "pf" (porteføljestatistikker)
    """
    # 1. Filtrer data: kun gyldige rækker med eom i dates, og vælg relevante kolonner.
    data_rel = data[(data['valid'] == True) & (data['eom'].isin(dates))][['id', 'eom', 'me', 'tr_ld1', 'pred_ld1']]
    data_rel = data_rel.sort_values(by=['id', 'eom'])
    
    # 2. Split data_rel efter eom.
    # Lav et dictionary med nøgler som eom og værdier som under-DataFrames.
    data_split = {d: df for d, df in data_rel.groupby('eom')}
    
    # 3. Beregn ønskede vægte (tpf_opt) for hver dato i dates.
    tpf_opt_list = []
    
    for d in dates:
        data_sub = data_split.get(d)
        if data_sub is None or data_sub.empty:
            continue
        # Hent de relevante id'er
        ids = data_sub['id'].tolist()
        
        # Udregn covariance-matricen for den aktuelle dato.
        # Forventningen er, at hvert element i cov_list har en metode create_cov, der returnerer sigma for de angivne ids.
        sigma = cov_list[d].create_cov(ids=ids)
        
        # Hent vektoren med prediktioner for den aktuelle dato.
        pred_vector = data_sub['pred_ld1'].values
        
        # Beregn de optimale vægte: w = (1/gam) * (sigma^-1 * pred_vector)
        weights = (1 / gam) * np.linalg.solve(sigma, pred_vector)
        
        # Lav et DataFrame med id, eom og den beregnede vægt.
        temp_df = data_sub[['id', 'eom']].copy()
        temp_df['w'] = weights
        tpf_opt_list.append(temp_df)
        
    # Kombinér resultaterne for alle datoer til ét DataFrame.
    if tpf_opt_list:
        tpf_opt = pd.concat(tpf_opt_list, ignore_index=True)
    else:
        tpf_opt = pd.DataFrame(columns=['id', 'eom', 'w'])
    
    # 4. Beregn de faktiske vægte vha. w_fun.
    # Her benyttes data_rel (vægtdata), wealth og de ønskede vægte (tpf_opt)
    tpf_w = w_fun(data_rel, dates, tpf_opt, wealth)
    
    # 5. Beregn porteføljestatistikker vha. pf_ts_fun og tilføj porteføljetypen "Markowitz-ML".
    tpf_pf = pf_ts_fun(tpf_w, data, wealth, gam)
    tpf_pf['type'] = "Markowitz-ML"
    
    # Output som en dictionary
    return {"w": tpf_w, "pf": tpf_pf}

# Eksempel på brug:
# Forudsæt at du har DataFrames: 'chars' (data), 'wealth', en liste/datoer 'dates_oos'
# samt en dictionary 'barra_cov' med covariance objekter og en parameter dictionary 'pf_set'
# tpf = tpf_implement(chars, cov_list=barra_cov, wealth=wealth, dates=dates_oos, gam=pf_set['gamma_rel'])


In [25]:

def create_cov(x, ids=None):
    """
    Beregner covariance matricen baseret på faktorloadings og ivol.

    Parametre:
      - x: dictionary med nøglerne 'fct_load', 'fct_cov' og 'ivol_vec'
      - ids: liste af id'er. Hvis None benyttes alle rækker.
    
    Returnerer:
      - cov_matrix: NumPy-array beregnet som (load @ fct_cov @ load.T + diag(ivol))
    """
    if ids is None:
        load = x['fct_load']
        ivol = x['ivol_vec']
    else:
        # Sørg for, at ids er strings
        ids = [str(i) for i in ids]
        load = x['fct_load'].loc[ids]
        ivol = x['ivol_vec'].loc[ids]
    
    fct_cov = x['fct_cov']
    if isinstance(fct_cov, pd.DataFrame):
        fct_cov = fct_cov.values

    load_array = load.values if isinstance(load, pd.DataFrame) else load
    ivol_array = ivol.values if isinstance(ivol, (pd.Series, pd.DataFrame)) else ivol

    cov_matrix = load_array.dot(fct_cov).dot(load_array.T) + np.diag(ivol_array)
    return cov_matrix

def tpf_implement(data, cov_list, wealth, dates, gam):
    """
    Beregner tangency portfolio weights og porteføljestatistikker.

    Parametre:
      - data: DataFrame med kolonnerne 'id', 'eom', 'me', 'tr_ld1', 'pred_ld1' og 'valid'
      - cov_list: dictionary med nøgler som datoer (som strings, fx '2019-12-31') og værdier,
                  hvor hvert element er en dictionary med nøglerne 'fct_load', 'fct_cov' og 'ivol_vec'
      - wealth: DataFrame med kolonnerne 'eom', 'mu_ld1' og 'wealth'
      - dates: liste over datoer (som pd.Timestamp eller strings)
      - gam: risikoaversion parameter

    Returnerer:
      - en dictionary med nøglerne "w" (de faktiske vægte) og "pf" (porteføljestatistikker)
    """
    # Filtrer data for gyldige rækker med eom i dates
    data_rel = data[(data['valid'] == True) & (data['eom'].isin(dates))][['id', 'eom', 'me', 'tr_ld1', 'pred_ld1']]
    data_rel = data_rel.sort_values(by=['id', 'eom'])
    
    # Opdel data_rel i grupper pr. eom
    data_split = {d: df for d, df in data_rel.groupby('eom')}
    
    tpf_opt_list = []
    
    for d in dates:
        # Konverter d til string, hvis d er en pd.Timestamp (cov_list-nøglerne forventes som strings)
        if isinstance(d, pd.Timestamp):
            d_key = d.strftime('%Y-%m-%d')
        else:
            d_key = d

        data_sub = data_split.get(d)
        if data_sub is None or data_sub.empty:
            continue

        ids = data_sub['id'].tolist()
        
        # Brug standalone funktionen create_cov her
        sigma = create_cov(cov_list[d_key], ids=ids)
        
        # Hent prediktioner for den aktuelle dato
        pred_vector = data_sub['pred_ld1'].values
        
        # Beregn de optimale vægte: w = (1/gam) * inv(sigma) * pred_vector
        weights = (1 / gam) * np.linalg.solve(sigma, pred_vector)
        
        temp_df = data_sub[['id', 'eom']].copy()
        temp_df['w'] = weights
        tpf_opt_list.append(temp_df)
    
    if tpf_opt_list:
        tpf_opt = pd.concat(tpf_opt_list, ignore_index=True)
    else:
        tpf_opt = pd.DataFrame(columns=['id', 'eom', 'w'])
    
    # Beregn de faktiske vægte vha. w_fun (forventes defineret et andet sted)
    tpf_w = w_fun(data_rel, dates, tpf_opt, wealth)
    # Beregn porteføljestatistikker vha. pf_ts_fun (forventes defineret et andet sted)
    tpf_pf = pf_ts_fun(tpf_w, data, wealth, gam)
    tpf_pf['type'] = "Markowitz-ML"
    
    return {"w": tpf_w, "pf": tpf_pf}

# Eksempel på brug:
# Forudsæt, at du har DataFrames: 'chars' (data), 'wealth', en liste 'dates_oos',
# en dictionary 'barra_cov' med covariance-dictionaries (nøgler som 'YYYY-MM-DD'),
# samt en parameter dictionary 'pf_set' med fx 'gamma_rel'.
#
# tpf = tpf_implement(data=chars, cov_list=barra_cov, wealth=wealth, dates=dates_oos, gam=pf_set['gamma_rel'])


In [26]:
result2 = tpf_implement(data=chars,cov_list=barra_cov, wealth=wealth, dates=dates_oos, gam=pf_set['gamma_rel'])


In [27]:
result2

{'w':          id        eom   w_start         w    tr_ld1    mu_ld1
 0     10104 2019-12-31  0.008246  0.041530       NaN       NaN
 1     10104 2020-01-31  0.041368  0.034603 -0.005238 -0.001351
 2     10104 2020-02-29  0.035469  0.072951 -0.056915 -0.079921
 3     10104 2020-03-31  0.082749 -0.025992 -0.022915 -0.138610
 4     10104 2020-04-30 -0.025256  0.027839  0.102269  0.134423
 ...     ...        ...       ...       ...       ...       ...
 8079  92655 2023-06-30  0.099256 -0.067100 -0.010165  0.069017
 8080  92655 2023-07-31 -0.068168 -0.058136  0.053142  0.036647
 8081  92655 2023-08-31 -0.055830 -0.013545 -0.058868 -0.019997
 8082  92655 2023-09-30 -0.015115 -0.059000  0.062111 -0.048173
 8083  92655 2023-10-31 -0.064529 -0.066500  0.061804 -0.029172
 
 [8084 rows x 6 columns],
 'pf':           inv   shorting   turnover         r         tc    eom_ret  \
 0   11.702437   5.487110  11.408339  0.019166  15.928972 2020-01-31   
 1   12.366381   5.821116   7.996968  0.001895   